# Apellidos de elite y financiamiento de campañas

Este notebook tiene el código de las regresiones calculadas y de las visualizaciones que se muestran en el artículo.

## Calculo de regresiones
Se utilizó R para calcular las regresiones. Las librerías de R están preparada para hacer exactamente lo que queríamos lograr.

In [1]:
! Rscript calculo_regresion.R

'Rscript' is not recognized as an internal or external command,
operable program or batch file.


## Visualizaciones
Con la regresión de todos los datos ya calculada creamos la visualización para los datos

In [4]:
import pandas as pd
import altair as alt
import numpy as np

### Transformar los datos

In [5]:
df = pd.read_csv('https://storage.googleapis.com/notas-blog-public/01%20-%20Elitismo%20Apellidos/aportes.csv')

In [8]:
df

,NOMBRE_APORTANTE,NOMBRE_CANDIDATO,PARTIDO,PACTO,DISTRITO,MONTO,APELLIDO_PATERNO,APELLIDO_MATERNO,NSE_PATERNO,FRECUENCIA_PATERNO,NSE_MATERNO,FRECUENCIA_MATERNO
0,NaN,PATRICIA ISABEL VALDERAS SILVA,IND PARTIDO COMUNISTA DE CHILE,APRUEBO DIGNIDAD,DISTRITO 21,20000.0,VALDERAS,SILVA,52.023958,216.0,53.782817,30064.0
1,ALEJANDRA ELIANA MORENO COX,ALVIN ANTONIO SALDAÑA MUÑOZ,INDEPENDIENTES,MOVIMIENTOS SOCIALES AUTONOMOS (D15),DISTRITO 15,20000.0,SALDAÑA,MUÑOZ,50.874588,1069.0,51.871280,60380.0
2,NaN,TERESA ADRIANA POBLETE PINOCHET,INDEPENDIENTES,INDEPENDIENTES Y MOVIMIENTOS SOCIALES DEL APRU...,DISTRITO 8,30000.0,POBLETE,PINOCHET,52.188148,9215.0,60.011292,1378.0
3,SANDRA PAOLA DIAZ CONEJEROS,TERESA ADRIANA POBLETE PINOCHET,INDEPENDIENTES,INDEPENDIENTES Y MOVIMIENTOS SOCIALES DEL APRU...,DISTRITO 8,10000.0,POBLETE,PINOCHET,52.188148,9215.0,60.011292,1378.0
4,PAOLA PATRICIA ITURRIETA LARA,JOSEFFE TAMARA CACERES TORRES,PARTIDO TRABAJADORES REVOLUCIONARIOS,PARTIDO DE TRABAJADORES REVOLUCIONARIOS,DISTRITO 12,5000.0,CACERES,TORRES,52.070924,11096.0,53.015663,20799.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9027,NaN,ESTEBAN ALEJANDRO VIELMA SALAZAR,REVOLUCION DEMOCRATICA,APRUEBO DIGNIDAD,DISTRITO 26,99.0,VIELMA,SALAZAR,54.437295,501.0,51.684693,11973.0
9028,PATRICIO ANDRES SAINZ REYES,FUAD CHAHIN VALENZUELA,PARTIDO DEMOCRATA CRISTIANO,LISTA DEL APRUEBO,DISTRITO 22,89.0,CHAHIN,VALENZUELA,84.200909,22.0,53.650710,25438.0
9029,PATRICIO ANDRES SAINZ REYES,PAZ ANASTASIADIS LE ROY,PARTIDO DEMOCRATA CRISTIANO,LISTA DEL APRUEBO,DISTRITO 7,34.0,LE,ROY,NaN,NaN,74.322692,13.0
9030,PATRICIO ANDRES SAINZ REYES,JORGE CORREA SUTIL,PARTIDO DEMOCRATA CRISTIANO,LISTA DEL APRUEBO,DISTRITO 6,13.0,CORREA,SUTIL,58.621734,7340.0,80.768421,19.0


In [6]:
df_montos = df.groupby(['NOMBRE_CANDIDATO', 'PACTO', 'DISTRITO', 'FRECUENCIA_PATERNO'], 
                       as_index=False).agg({'MONTO': sum,
                                            'NSE_PATERNO': 'first',
                                            'NSE_MATERNO': 'first'})
df_montos = df_montos[~df_montos['NSE_MATERNO'].isna()]

In [7]:
grupos_lista = ['VAMOS POR CHILE', 'LISTA DEL APRUEBO', 'APRUEBO DIGNIDAD']
df_montos['PACTO'] = df_montos['PACTO'].apply(lambda x: x if x in grupos_lista else 'OTRO')

### Crear los elementos de la visualización

In [6]:
nse_paterno_domain = (30, 100)
pactos_color_scale = alt.Scale(domain=grupos_lista + ['OTRO'], range=['#277DA1', '#F9C74F', '#F94144', '#90BE6D'])
selection = alt.selection_multi(fields=['PACTO'])
color = alt.condition(selection,
                      alt.Color('PACTO:N', legend=None, 
                                scale=pactos_color_scale),
                      alt.value('#EBEBEB'))
x = alt.X('NSE_PATERNO', title='Nivel socioeconómico apellido del padre', scale=alt.Scale(domain=nse_paterno_domain))
y = alt.Y('MONTO', axis=alt.Axis(title='Total de aportes recibidos', format='$.2s'))

#### Leyenda interactiva

In [7]:
legend = alt.Chart(df_montos).mark_point(filled=True, size=80).encode(
    y=alt.Y('PACTO:N', axis=alt.Axis(orient='right', title='Pacto')),
    color=color
).add_selection(
    selection
)


#### Regresión todos las listas
Esta fue calculada con R

In [8]:
nse_paterno = np.linspace(*nse_paterno_domain)
coefs = pd.read_csv('coefs.csv', index_col=0).T
monto = coefs.loc['x', "(overall_intercept)"] + coefs.loc['x', 'NSE_PATERNO'] * nse_paterno 

regresion = (alt.Chart(pd.DataFrame({'NSE_PATERNO': nse_paterno , 'MONTO': monto}))
             .mark_line(color='black')
             .encode(x=x, y=y))

#### Regresión por lista

In [9]:
regresion_pactos = (alt.Chart(df_montos)
                    .mark_point().encode(
                        x=x,
                        y=y,
                        color=color)
                    .transform_regression('NSE_PATERNO', 'MONTO', 
                                          groupby=['PACTO'], extent=nse_paterno_domain)
                    .mark_line())

#### Scatter de los candidatos

In [10]:
scatter = alt.Chart(df_montos).mark_point(clip=True, filled=True, size=80, opacity=0.8).encode(
    x=x,
    y=y,
    color=color,
    tooltip=[alt.Tooltip('NOMBRE_CANDIDATO:N', title='Nombre', type='nominal'),
             alt.Tooltip('PACTO:N', title='Pacto', type='nominal'),
             alt.Tooltip('MONTO:Q', title='Monto en pesos', format=',', type='quantitative'),
             alt.Tooltip('FRECUENCIA_PATERNO:Q', title='Frecuencia apellido paterno', format=',', type='quantitative')])

In [11]:
alt.renderers.set_embed_options(formatLocale={
  "decimal": ",",
  "thousands": ".",
  "grouping": [3],
  "currency": ["$", "M"]
})

chart = (scatter
 + regresion_pactos
 + regresion)

chart_legend = chart.interactive()  | legend
chart_legend.save('Nota 01 - Vis 01.html')
chart_legend 

alt.HConcatChart(...)

In [14]:
bar_chart_nse = (alt.Chart(df_montos).mark_bar().encode(
    y=alt.Y('PACTO',sort='-x', title=''),
    color= alt.Color('PACTO:N', legend=None, 
                                scale=pactos_color_scale),
    x=alt.X("mean(NSE_PATERNO)", title='Nivel socioeconómico del padre (promedio)')
))
bar_chart_nse.save('Nota 01 - Vis 02.html')
bar_chart_nse

alt.Chart(...)

In [13]:
bar_chart_aportes = (alt.Chart(df_montos).mark_bar().encode(
    y=alt.Y('PACTO',sort='-x', title=''),
    color= alt.Color('PACTO:N', legend=None, 
                                scale=pactos_color_scale),
    x=alt.X("mean(MONTO)",axis=alt.Axis(title='Total de aportes recibidos', format='$.2s'))
))
bar_chart_aportes.save('Nota 01 - Vis 03.html')
bar_chart_aportes

alt.Chart(...)